# This is weeks 3 peer graded assignment

## 1) import the data from the wikipedia page and clean it up

In [1]:
pip install beautifulsoup4


The following command must be run outside of the IPython shell:

    $ pip install beautifulsoup4

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [2]:
pip install lxml


The following command must be run outside of the IPython shell:

    $ pip install lxml

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [3]:
pip install requests


The following command must be run outside of the IPython shell:

    $ pip install requests

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [4]:
from bs4 import BeautifulSoup
import requests

Read all data as html from the webpage https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

Get the required data were in a table with class 'wikitable sortable'. Find it

In [6]:
tableHTML = soup.find('table', class_ = 'wikitable sortable')

Each row is in tr block and each entry is in td block. Use a nested loop to obtain a list of all entries

In [7]:
table = []
for row in tableHTML.find_all('tr'):
    
    for entry in row.find_all('td'):
        table.append(entry.text)

Seperate the list every three entries. ie, every 3 entries make a row

In [8]:
def devideList(List, size):
    for i in range(0, len(List), size):  
        yield List[i:i + size]

In [9]:
table = list(devideList(table, 3))

In [10]:
import pandas as pd

Change the list to a DF

In [11]:
neighbourhood = pd.DataFrame(table, columns = ["Postcode", "Borough", "Neighbourhood"])
neighbourhood.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


Remove the '\n' in the entries

In [12]:
neighbourhood['Neighbourhood'].replace(regex=True,inplace=True,to_replace='\n',value=r'')
neighbourhood.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove the rows with Not Assigned in Borough column

In [13]:
neighbourhood.drop(neighbourhood.loc[neighbourhood['Borough']=='Not assigned'].index, inplace=True)
neighbourhood.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Group neghbourhoods of the same postal code together. Then change them to strings.

In [14]:
x = neighbourhood.groupby('Postcode')['Neighbourhood'].apply(list)
x = x.reset_index(name='Neighbourhood')

In [15]:
x.head()

,Postcode,Neighbourhood
0,M1B,"[Rouge, Malvern]"
1,M1C,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,"[Guildwood, Morningside, West Hill]"
3,M1G,[Woburn]
4,M1H,[Cedarbrae]


In [16]:
x['Neighbourhood1'] = x.Neighbourhood.apply(', '.join)
x.head()

,Postcode,Neighbourhood,Neighbourhood1
0,M1B,"[Rouge, Malvern]","Rouge, Malvern"
1,M1C,"[Highland Creek, Rouge Hill, Port Union]","Highland Creek, Rouge Hill, Port Union"
2,M1E,"[Guildwood, Morningside, West Hill]","Guildwood, Morningside, West Hill"
3,M1G,[Woburn],Woburn
4,M1H,[Cedarbrae],Cedarbrae


In [17]:
x = pd.merge(x,neighbourhood, on='Postcode')
x.head()

,Postcode,Neighbourhood_x,Neighbourhood1,Borough,Neighbourhood_y
0,M1B,"[Rouge, Malvern]","Rouge, Malvern",Scarborough,Rouge
1,M1B,"[Rouge, Malvern]","Rouge, Malvern",Scarborough,Malvern
2,M1C,"[Highland Creek, Rouge Hill, Port Union]","Highland Creek, Rouge Hill, Port Union",Scarborough,Highland Creek
3,M1C,"[Highland Creek, Rouge Hill, Port Union]","Highland Creek, Rouge Hill, Port Union",Scarborough,Rouge Hill
4,M1C,"[Highland Creek, Rouge Hill, Port Union]","Highland Creek, Rouge Hill, Port Union",Scarborough,Port Union


In [18]:
x.drop(columns=['Neighbourhood_x','Neighbourhood_y'], inplace = True)
x = x.rename(columns={'Neighbourhood1':'Neighbourhood'})
x.head()

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1B,"Rouge, Malvern",Scarborough
2,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
4,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough


In [19]:
x = x.drop_duplicates()
x.reset_index(drop=True, inplace = True)
x.head()

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


Rearrange Df to find the get the final df and show its head and shape.

In [20]:
FinalDf = x[['Postcode','Borough','Neighbourhood']]
FinalDf.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
FinalDf.shape

(103, 3)

## 2) get all the required magnitudes and longitudes

In [33]:
!conda install -c conda-forge geocoder
import geocoder # import geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [37]:
latAndLng = []
for postcode in FinalDf['Postcode']:
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(postcode))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    latAndLng.append([latitude,longitude])

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [43]:
dfLatAndLng = pd.DataFrame(latAndLng, columns = ['Latitude' , 'Longitude']) 
FinalDf = pd.concat([FinalDf, dfLatAndLng],axis=1)

Display the final DF that will be used during clustering

In [44]:
FinalDf.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726276,-79.263625
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713054,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.724235,-79.227925
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696770,-79.259967
